In [1]:
## TOPIC Modelling

In [2]:
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim
from nltk.stem import LancasterStemmer, WordNetLemmatizer, PorterStemmer
# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Import Dataset
df= pd.read_csv('/Users/vibhuverma/Desktop/CLASSWORK/BUSINESS PRACTICUM/Anti-Corruption/Data Provided/USAID_Project_Data/USAID_Anticorruption_Projects_Database.csv')
df = df.drop(['rfp_rftop','final_res','mind_eval','final_eval','audit','interim_rep','addl_docs','addl_info'],axis=1)

In [4]:
# Convert to list
data = df.proj_desc.values.tolist()

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['The five components of this program seek to take what already exists and '
 'improve upon it, implement it and find ways to ensure that the progress '
 'achieved is sustainable over the long term. The project will help the '
 'government of the Islamic Republic of Afghanistan strengthen its High Office '
 'of Oversight (HOO) - making it a strong, effective institution that is able '
 'to lead, monitor, coordinate and report on efforts to combat corruption '
 'across the country. The project will work and implement a program of '
 'institutional development and sustainability. It will also support the '
 'office in carrying out its priority responsibilities in asset registration '
 'and verification, complaints management and case tracking, and coordination '
 'and monitoring of anticorruption performance across other government '
 'agencies.']


In [5]:
#tokenize and clean up using gensim 

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))


In [6]:
#lemmatization

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:2])

['component program seek take already exist improve implement find way ensure progress achieve sustainable long term project will help government strengthen high office oversight hoo make strong effective institution able lead monitor coordinate report effort combat corruption country project will work implement program institutional development sustainability will also support office carry priority asset registration verification complaint management case tracking coordination monitoring anticorruption performance other government agency', 'overall goal program support rule law strategy specifically pillar improve justice sector reduce corruption pillar build leadership justice sector civil society support pillar contractor attempt develop capacity judiciary law school pillar contractor work raise public awareness encourage citizen resolve dispute formal justice sector program divide component capacity building judiciary capacity building court management system capacity building facu

In [7]:
## TFIDF


vectorizer = TfidfVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             ngram_range=(1,3)             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [8]:
#Checking the sparcity

# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  9.346462864981383 %


In [9]:
## BASE MODEL


# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=20,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=42,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(learning_method='online', n_components=20, n_jobs=-1,
                          random_state=42)


In [10]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))

# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -11400.430736159025
Perplexity:  1799.893102521426
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 20,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 42,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [11]:
## HYPERPARAMETER TUNING

# Define Search Param
search_params = {'n_components': np.arange(5,30).tolist(), 'learning_decay': [.3,.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)

GridSearchCV(estimator=LatentDirichletAllocation(),
             param_grid={'learning_decay': [0.3, 0.5, 0.7, 0.9],
                         'n_components': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,
                                          16, 17, 18, 19, 20, 21, 22, 23, 24,
                                          25, 26, 27, 28, 29]})

In [12]:
## BEST TOPIC MODEL


# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))


Best Model's Params:  {'learning_decay': 0.7, 'n_components': 5}
Best Log Likelihood Score:  -2488.476190070669
Model Perplexity:  895.3521283172437


In [13]:
### DOMINANT TOPIC IN EACH DOCUMENT


# Create Document - Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(data))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics




,Topic0,Topic1,Topic2,Topic3,Topic4,dominant_topic
Doc0,0.030000,0.360000,0.030000,0.030000,0.560000,4
Doc1,0.030000,0.030000,0.080000,0.030000,0.820000,4
Doc2,0.050000,0.050000,0.050000,0.050000,0.810000,4
Doc3,0.050000,0.050000,0.130000,0.630000,0.150000,3
Doc4,0.040000,0.040000,0.040000,0.040000,0.860000,4
Doc5,0.760000,0.150000,0.030000,0.030000,0.030000,0
Doc6,0.030000,0.690000,0.030000,0.220000,0.030000,1
Doc7,0.790000,0.040000,0.090000,0.040000,0.040000,0
Doc8,0.850000,0.040000,0.040000,0.040000,0.040000,0
Doc9,0.880000,0.030000,0.030000,0.030000,0.030000,0


In [14]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
1      150.222153  -72.269554       1        1  36.181964
4       51.163364 -159.968018       2        1  31.346305
0      -73.144554   21.523815       3        1  18.118348
3      -66.410133 -110.606438       4        1   8.789323
2       54.417324   12.443444       5        1   5.564060, topic_info=                 Term       Freq      Total Category  logprob  loglift
180             local  15.000000  15.000000  Default  30.0000  30.0000
25         assessment   5.000000   5.000000  Default  29.0000  29.0000
62          community   7.000000   7.000000  Default  28.0000  28.0000
134            health   5.000000   5.000000  Default  27.0000  27.0000
182  local government   7.000000   7.000000  Default  26.0000  26.0000
..                ...        ...        ...      ...      ...      ...
233           program   0.919161  16.435052   Topic5  -4.5226   0.0051
90           economic   0.652064   7.725093   Topic5  -4.8659   0.4168
277            sector   0.619350   9.744580   Topic5  -4.9173   0.1330
122              goal   0.540857   8.091230   Topic5  -5.0529   0.1835
86             design   0.525012   7.004924   Topic5  -5.0826   0.2979

[267 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
0         1  0.360893      ability
0         3  0.360893      ability
0         5  0.360893      ability
3         2  0.477995  accountable
3         3  0.477995  accountable
...     ...       ...          ...
322       3  0.244153         work
322       4  0.122076         work
323       2  0.392013         year
323       4  0.392013         year
323       5  0.392013         year

[428 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 1, 4, 3])

In [15]:
### TOPIC KEYWORDS ###

# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)

# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames

# View
df_topic_keywords.head()

,ability,access,accountability,accountable,achieve,action,activity,actor,address,administration,...,training,transition,transparency,transparency accountability,transparent,unit,use,woman,work,year
Topic0,0.742740,0.357712,2.014789,2.197499,0.228813,0.202396,1.259407,0.200082,0.670297,0.684418,...,0.984742,0.223109,2.303592,1.400707,2.504602,0.806785,0.321258,0.201443,2.275515,0.321142
Topic1,1.353326,2.323023,2.957657,0.201836,2.991159,1.072688,1.870168,1.453522,1.601738,1.953674,...,4.911125,0.473495,3.703904,2.108696,1.485232,1.233165,1.904221,2.932325,2.685997,0.234326
Topic2,0.923948,0.231724,0.200308,0.200074,0.202523,0.201475,0.530049,0.946431,0.200755,0.315814,...,0.200611,0.200189,0.245821,0.200493,0.200148,0.202472,0.330514,0.703515,0.270917,1.134494
Topic3,0.200908,0.206917,0.200216,0.311654,0.233166,0.203430,0.810209,0.220387,1.122148,0.200065,...,1.008408,1.704474,0.200263,0.200337,0.497820,0.200091,0.960732,1.170167,1.793065,0.909704
Topic4,0.254126,3.650928,3.945875,2.174659,1.046455,1.476411,3.087182,1.151760,2.402097,3.080630,...,1.779783,1.077513,4.887591,1.602066,1.859453,0.507090,1.789145,0.220694,2.968995,0.764267


In [16]:
### TOP 15 KEYWORDS FOR EACH TOPIC ###

# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,local,local government,municipality,municipal,government,governance,citizen,service,local governance,participation,level,capacity,development,citizen participation,official
Topic 1,improve,government,project,support,technical,assistance,public,program,business,reform,technical assistance,trade,policy,investment,economic
Topic 2,assessment,health,community,reduce corruption,right,base,identify,opportunity,corruption,tax,management,program,resource,establishment,social
Topic 3,democratic,political,health,democracy,program,community,afghan,governance,strengthen,aim,network,support,conflict,advocacy,work
Topic 4,corruption,justice,judicial,project,legal,strengthen,law,court,increase,society,program,rule law,public,civil society,rule


In [17]:
### Document Clustering with topic model as features ###
tt_matrix = best_lda_model.components_

In [18]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=5, random_state=0)
km.fit_transform(lda_output)
cluster_labels = km.labels_
cluster_labels = pd.DataFrame(cluster_labels, columns=['ClusterLabel'])
pd.concat([df, cluster_labels], axis=1)

,uuid,region,country,proj_nm,st_date,end_date,awd_amount,awd_value,implementer,sector,intervention,intervention_type,proj_desc,proj_res,corruption,stability,ClusterLabel
0,1,South and Central Asia (SCA),Afghanistan,Assistance for Afghanistan's Anti-Corruption A...,2010,2013,e) $25 M-$50 M,26642989,Management Systems International (MSI),"Independent Agencies, Civil Society","Corruption Prevention, Public Awareness","Awareness, Accountability",The five components of this program seek to ta...,The program developed a working relationship w...,High Corruption,Unstable,1
1,2,South and Central Asia (SCA),Afghanistan,Rule of Law Stabilization Program - Formal Com...,2010,2012,e) $25 M-$50 M,33700000,TetraTech ARD,Rule of Law,"Court Administration, Professional Legal Educa...","Transparency, Awareness",The overall goal of the program was to support...,The project set up legal education training fo...,Very High Corruption,Very Unstable,1
2,3,South and Central Asia (SCA),Afghanistan,Afghanistan Rule of Law Stabilization Project ...,2011,2012,c) $5 M-$10 M,10000000,"Checchi and Company Consulting, Inc.",Rule of Law,Access to Justice,"Awareness, Accountability",The project focused on strengthening and impro...,"The project reached 37,583 participants. In Ph...",Very High Corruption,Very Unstable,1
3,4,South and Central Asia (SCA),Afghanistan,Political Party Assessment,2012,2012,a) <$1 M,NaN,"Checchi and Company Consulting, Inc.",Elections and Political Processes,Assessment,Assessment,The Political Party Assessment team aimed to r...,The assessment team found that the development...,Very High Corruption,Very Unstable,3
4,30,Europe and Eurasia (E&E),Albania,Justice Sector Strengthening Project (JuST),2010,2015,c) $5 M-$10 M,9012613,Chemonics,"Rule of Law, Civil Society","Legal/Institutional Strengthening/Reform, Cour...","Transparency, Accountability, Awareness",The Justice Sector Strengthening Project (JuST...,"As of September 30, 2012, the project installe...",High Corruption,Unstable,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,311,Middle East and North Africa (MENA),Yemen,Responsive Governance Project (RGP),2010,2015,e) $25 M-$50 M,43000000,Counterpart International,"Civil Society, Legislature",Legislative/Institutional Strengthening/Reform...,"Enforcement, Citizen/Media/Business Engagement...",The Yemen Responsive Governance Project (RGP) ...,"To date, the RGP has assisted in crafting Yeme...",High Corruption,Very Unstable,1
311,312,Middle East and North Africa (MENA),Yemen,Enhancing Government Effectiveness Project (EGE),2007,2010,b) $1 M-$5 M,2939048,Management Systems International (MSI),Governance,"Financial Management Systems, Public Information","Transparency, Accountability",The Enhancing Government Effectiveness (EGE) i...,This project contributed to and improved capac...,High Corruption,Very Unstable,0
312,313,Middle East and North Africa (MENA),Yemen,Basic Health Services Project (BHS),2006,2010,d) $10 M-$25 M,21610314.78,Pathfinder International,"Healthcare, Governance","Service Delivery, Internal Controls, Procurement","Accountability, Awareness, Participation",The Basic Health Services (BHS) project was d...,The renovations and refurbishing of health fac...,High Corruption,Very Unstable,0
313,314,Sub-Saharan Africa (SSA),Zambia,Zambia Threshold Project (ZTP),2006,2008,d) $10 M-$25 M,22735000,Chemonics,"Rule of Law, Private Sector","Trade Reforms, Business Environment, Corruptio...","Transparency, Participation",The program was designed to reduce administrat...,The program promoted greater transparency and ...,High Corruption,Unstable,0
